Every Corpus has three main components: Conversations, Utterances, and Speakers.

See this link for more in depth tutorial: https://convokit.cornell.edu/documentation/tutorial.html

In [3]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np

## Download Data

In [1]:
# from convokit import Corpus, download
# corpus = Corpus(filename=download("movie-corpus"))

In [4]:
# corpus.print_summary_stats()

In [5]:
# speakers_df = corpus.get_speakers_dataframe()

In [6]:
# speakers_df.head()  

In [7]:
# convo_df = corpus.get_conversations_dataframe()

In [8]:
# convo_df.head()

In [2]:
# utt_df = corpus.get_utterances_dataframe()

In [3]:
# utt_df.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.movie_id,meta.parsed,vectors
id,,,,,,,,
L1045,None,They do not!,u0,L1044,L1044,m0,"[{'rt': 1, 'toks': [{'tok': 'They', 'tag': 'PR...",[]
L1044,None,They do to!,u2,None,L1044,m0,"[{'rt': 1, 'toks': [{'tok': 'They', 'tag': 'PR...",[]
L985,None,I hope so.,u0,L984,L984,m0,"[{'rt': 1, 'toks': [{'tok': 'I', 'tag': 'PRP',...",[]
L984,None,She okay?,u2,None,L984,m0,"[{'rt': 1, 'toks': [{'tok': 'She', 'tag': 'PRP...",[]
L925,None,Let's go.,u0,L924,L924,m0,"[{'rt': 0, 'toks': [{'tok': 'Let', 'tag': 'VB'...",[]


In [11]:
# utt_df.to_csv('utterances.csv', index=False)
# speakers_df.to_csv('speakers.csv', index=False)
# convo_df.to_csv('conversations.csv', index=False)

## Pred DF

In [4]:
utt_df = pd.read_csv("utterances.csv")

In [5]:
pred_df = pd.DataFrame(columns=['utterance text', 'speaker', 'conversation_id'] + list(range(28)))
pred_df['utterance text'] = utt_df['text']
pred_df['speaker'] = utt_df['speaker']  
pred_df['conversation_id'] = utt_df['conversation_id']

In [6]:
pred_df = pred_df.dropna(subset=['utterance text'])

In [7]:
pred_df

,utterance text,speaker,conversation_id,0,1,2,3,4,5,6,...,18,19,20,21,22,23,24,25,26,27
0,They do not!,u0,L1044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,They do to!,u2,L1044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I hope so.,u0,L984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,She okay?,u2,L984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Let's go.,u0,L924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304708,Lord Chelmsford seems to want me to stay back ...,u9030,L666369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304709,I'm to take the Sikali with the main column to...,u9034,L666369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304710,"Your orders, Mr Vereker?",u9030,L666369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304711,"Good ones, yes, Mr Vereker. Gentlemen who can ...",u9030,L666256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# pred_df.to_csv("predictions.csv")

In [17]:
# utt_text = np.array(pred_df['utterance text'])

In [18]:
# lengths = [len(s.split()) if type(s) == str else len(str(s).split()) for s in utt_text ]

In [19]:
# # Plot the histogram
# plt.hist(lengths, bins=range(1, max(lengths) +2 ) )
# plt.xlabel('String Length')
# plt.ylabel('Frequency')
# plt.title('Histogram of String Lengths')
# plt.show()